In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from tqdm.notebook import tqdm

apidata = pd.read_csv("../Dataset/enrichedBookData.csv", sep="|", error_bad_lines=False)
book = pd.read_csv("../Dataset/items.csv", sep="|")

C:\Users\rmaql\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
book

,itemID,title,author,publisher,main topic,subtopics
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH]
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]"
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]"
3,40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]"
4,46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]"
...,...,...,...,...,...,...
78025,37678,Timeless Fairy Tales,"Brothers Grimm, Marie-Catherine Baroness D'Aulnoy",MEDIAMORPHOSIS,YFA,[YFJ]
78026,68688,Demon Games,A. Witt Timothy a. Witt,iUniverse,FMB,[]
78027,57291,Lori and the Lion's Den,A. M. Glass,Xlibris,YFU,[]
78028,78130,The Everywhere Armchair,Ersila Bee,ELOQUENT BOOKS,YFC,"[YFG,YFH]"


In [4]:
apidata.sort_values('titleQueried')
apidata.head()

,titleQueried,titleFound,author,language,description,isbn
0,Princess Poppy: The Big Mix Up,The Big Mix-up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.780553e+12
1,Princess Poppy: The Big Mix Up,Princess Poppy: The Big Mix Up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781409e+12
2,Princess Poppy: The Big Mix Up,Princess Poppy: The Haunted Holiday,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781409e+12
3,Princess Poppy: The Big Mix Up,Princess Poppy: The Fashion Princess,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781407e+12
4,Princess Poppy: The Big Mix Up,Princess Poppy: The Rescue Princess,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781407e+12


In [5]:
apidata.isnull().sum()

titleQueried        0
titleFound      11321
author          30079
language        11700
description     73095
isbn            46067
dtype: int64

In [6]:
apidata.shape

(160769, 6)

In [7]:
# remove row with no author
processedApi = apidata[apidata['author'].notna()]

In [8]:
processedApi.isnull().sum()

titleQueried        0
titleFound          0
author              0
language           32
description     44601
isbn            24072
dtype: int64

In [9]:
# make a dataframe w/ books in common between original data and apidata
mask = processedApi['titleQueried'].isin(book['title'])
booksInCommon,processedApi_left = processedApi[mask],processedApi[~mask]

In [10]:
booksInCommon = booksInCommon.sort_values('titleQueried')
booksInCommon['key:title-author'] = ((booksInCommon['titleQueried'] + ' '+ booksInCommon['author']).astype(str)).str.lower()
booksInCommon = booksInCommon.drop_duplicates('key:title-author', keep='first')
# booksInCommon.to_excel('check2.xlsx')

In [11]:
book['key:title-author'] = ((book['title'] + ' '+ book['author']).astype(str)).str.lower()
book = book.sort_values('title')
book

,itemID,title,author,publisher,main topic,subtopics,key:title-author
34478,26966,TRIPION minaccia dallo spazio,Stefano Grimaldi,Lulu.com,FL,[],tripion minaccia dallo spazio stefano grimaldi
66492,24125,The Ultimate Vehicle Colouring Book for Kids,Chetna .,Westland Publications Limited,YBG,[],the ultimate vehicle colouring book for kids ...
31055,64846,!!,NaN,AUTHORHOUSE,FM,[],nan
74481,64745,"#4 Happy Birthday, Mallory!",Laurie Friedman,DARBY CREEK,YF,[5HKA],"#4 happy birthday, mallory! laurie friedman"
2806,7780,#Basteln for Future,"Susanne Pypke, Naturschutzjugend NAJU",Frech Verlag GmbH,WF,"[5AF,Y]","#basteln for future susanne pypke, naturschutz..."
...,...,...,...,...,...,...,...
61016,21473,Übernatürliche Irreführung (Sasha Urban Serie 5),"Dima Zales, Anna Zaires",MOZAIKA LLC,FMX,[],übernatürliche irreführung (sasha urban serie ...
56197,35275,Überschneidung zweier untersschiedlicher Welten,WErner Röschl,Books on Demand,FD,"[FD,FM]",überschneidung zweier untersschiedlicher welte...
56191,50552,Übersinnlicher Retter,Tina Folsom,Duboce Park Press,FRT,[],übersinnlicher retter tina folsom
20130,5665,Übungsheft Handlettering: Übungsbuch Für Hand ...,"Sunshine Handlettering, Handlettering Heft In ...",INDEPENDENTLY PUBLISHED,YNV,[],übungsheft handlettering: übungsbuch für hand ...


In [12]:
# check similarity
dftemp = DataFrame (range(0),columns=list(booksInCommon.columns))
threshold = 80
threshold2 = 50  ## Conservative standards to drop unrelated apidata
j_end = 0

for i in tqdm(range(len(book))):    
    for j in range(j_end,len(booksInCommon)):
        # if titles from two data are the same
        if book['title'].iloc[i] == booksInCommon.get('titleQueried').iloc[j]:
            j_end += 1
            # if similarity between key:title-author > threshold
            if (fuzz.token_sort_ratio(booksInCommon.get('key:title-author').iloc[j], book.get('key:title-author').iloc[i])) > threshold:
                # if similarity between titleQueried and titleFound  > threshold2, then save it as a dataframe
                if (fuzz.token_sort_ratio(booksInCommon.get('titleQueried').iloc[j], booksInCommon.get('titleFound').iloc[j])) > threshold2:
                    dftemp = dftemp.append(booksInCommon.iloc[j])     
        else:
            j_end = j
            break;

In [13]:
dftemp

,titleQueried,titleFound,author,language,description,isbn,key:title-author
151865,#Instacat,#Instacat,Rae Earl,de,"Instagram, Eltern und andere Katastrophen ... ...",9.783846e+12,#instacat rae earl
4151,#Wir bleiben zuhause Kindermitmachbuch,#Wir bleiben zuhause Kindermitmachbuch,Frechverlag,de,NaN,9.783772e+12,#wir bleiben zuhause kindermitmachbuch frechve...
152172,'SIKARAN SHIELD LORD',Sikaran Shield Lord,Joanna Z. Ray,en,NaN,9.781858e+12,'sikaran shield lord' joanna z. ray
152173,'SIKARAN SHIELD LORD',SIKARAN LIGHT,Joanna Ray,en,The Sikarans and humans are developing many in...,9.781327e+12,'sikaran shield lord' joanna ray
117696,(Un)arranged Marriage,(Un)arranged Marriage,Bali Rai,en,MANNY WANTS TO BE A FOOTBALLER. OR A POP STAR....,9.781446e+12,(un)arranged marriage bali rai
...,...,...,...,...,...,...,...
32577,"»Super«, schreit der Frieder, und die Oma kich...","»Super«, schreit der Frieder, und die Oma kich...",Gudrun Mebs,de,Der fünfte Band rund um Frieder und Oma und Fr...,9.783734e+12,"»super«, schreit der frieder, und die oma kich..."
121474,Über die Grenze,Über die Grenze,Maja Lunde,de,Norwegen unter deutscher Besatzung 1942. Der z...,9.783825e+12,über die grenze maja lunde
121133,Über kurz oder lang,Über kurz oder lang,Marie-Aude Murail,de,Der neue großartige Roman von Marie-Aude Murai...,9.783104e+12,über kurz oder lang marie-aude murail
121135,Über kurz oder lang,Über kurz oder lang,"Marie-Aude Murail, Elinor Matt",de,NaN,9.783941e+12,"über kurz oder lang marie-aude murail, elinor ..."


In [14]:
# check whether they are from collection, the same book, or different book
n = 8

dftemp2 = dftemp.copy()
dftemp2['titleQueried'] = (dftemp2['titleQueried']).astype(str).str.lower()
dftemp2['titleFound'] =(dftemp2['titleFound']).astype(str).str.lower()
dftemp2["titleQueried"] = dftemp2["titleQueried"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
dftemp2["titleFound"] = dftemp2["titleFound"].str.replace(pat=r'[^\w]', repl=r'', regex=True)

diffList = []
sameList = []
collectionList = []

for i in tqdm(range(len(dftemp2))):
    # if title is not the same --> need to further check
    if dftemp2.get('titleQueried').iloc[i] != dftemp2.get('titleFound').iloc[i]:
        ## if titleQueried and titleFound similarity < 97 --> need to further check 
        if fuzz.token_sort_ratio(dftemp2.get('titleQueried').iloc[i], dftemp2.get('titleFound').iloc[i]) < 97:
            ### if firt n-letter similarity > 80 --> they are from a collection
            if (fuzz.token_sort_ratio(dftemp2.get('titleQueried').iloc[i][:n], dftemp2.get('titleFound').iloc[i][:n]) > 80):
                collectionList.append(dftemp.iloc[i])
            elif fuzz.token_sort_ratio(dftemp2.get('titleQueried').iloc[i], dftemp2.get('titleFound').iloc[i]) > 70:
                collectionList.append(dftemp.iloc[i])
            else:
                diffList.append(dftemp.iloc[i])
        ## if titleQueried and titleFound similarity > 97, they are the same book 
        else:
            sameList.append(dftemp.iloc[i])
    # if titme is the same, they are the same book
    else:
        sameList.append(dftemp.iloc[i])
            
collectionDf = pd.DataFrame(collectionList)
diffDf = pd.DataFrame(diffList)
sameDf = pd.DataFrame(sameList)

In [15]:
#fuzz.token_sort_ratio('Alex Rider 2: Stormbreaker', 'Alex Rider 1: Stormbreaker')

### collection: collection(1) same book(2)

In [16]:
# collectionDf, diffDf, sameDf
diffDf.to_excel('diffDf.xlsx')
diffDf

,titleQueried,titleFound,author,language,description,isbn,key:title-author
74150,1000 Gefahren am Katastrophentag,Katastrophentag,Edward Packard,en,The reader's choices determine the course of a...,9.783474e+12,1000 gefahren am katastrophentag edward packard
137495,1000 Gefahren bei den Dinosauriern,Fußballpokal der 1000 Gefahren,Fabian Lenk,de,"Hier entscheidest du allein, wie dein Abenteue...",9.783473e+12,1000 gefahren bei den dinosauriern fabian lenk
103369,1000 Gefahren im alten Rom,Das Fußballspiel der 1000 Gefahren,Fabian Lenk,de,"Hier entscheidest du allein, wie dein Abenteue...",9.783473e+12,1000 gefahren im alten rom fabian lenk
101096,1000 Gefahren in der Wildnis,Fußballpokal der 1000 Gefahren,Fabian Lenk,de,"Hier entscheidest du allein, wie dein Abenteue...",9.783473e+12,1000 gefahren in der wildnis fabian lenk
7415,111 x Papierfalten - Ostern,Bastel-Kids - Papierfalten im Quadrat,Thade Precht,de,NaN,9.783864e+12,111 x papierfalten - ostern thade precht
...,...,...,...,...,...,...,...
154759,Zoe's Rescue Zoo: The Wild Wolf Pup,The Wild Wolf Pup,Amelia Cobb,en,Zoe loves living at her uncle's rescue zoo bec...,9.780858e+12,zoe's rescue zoo: the wild wolf pup amelia cobb
21723,Zwei Wege in den Sommer,Wer wir sein könnten,Robert Habeck,de,"„Wie wir sprechen, entscheidet darüber, wer wi...",9.783462e+12,zwei wege in den sommer robert habeck
91233,Zwischen den Welten 01 - Daughter of Smoke and...,Daughter of Smoke and Bone,Laini Taylor,de,"Es war einmal, da verliebten sich ein Engel un...",9.783104e+12,zwischen den welten 01 - daughter of smoke and...
95826,moses. - black stories - Fantasy Movie Edition,Black stories - real crime edition,Tanja Mues,en,NaN,9.783898e+12,moses. - black stories - fantasy movie edition...


In [17]:
collectionDf['collection'] = 1
collectionDf
collectionDf.to_excel('collectionDf.xlsx')

In [18]:
sameDf['collection'] = 2
sameDf

,titleQueried,titleFound,author,language,description,isbn,key:title-author,collection
151865,#Instacat,#Instacat,Rae Earl,de,"Instagram, Eltern und andere Katastrophen ... ...",9.783846e+12,#instacat rae earl,2
4151,#Wir bleiben zuhause Kindermitmachbuch,#Wir bleiben zuhause Kindermitmachbuch,Frechverlag,de,NaN,9.783772e+12,#wir bleiben zuhause kindermitmachbuch frechve...,2
152172,'SIKARAN SHIELD LORD',Sikaran Shield Lord,Joanna Z. Ray,en,NaN,9.781858e+12,'sikaran shield lord' joanna z. ray,2
117696,(Un)arranged Marriage,(Un)arranged Marriage,Bali Rai,en,MANNY WANTS TO BE A FOOTBALLER. OR A POP STAR....,9.781446e+12,(un)arranged marriage bali rai,2
4931,... und dann kam Joselle,... und dann kam Joselle,"Kevin Henkes, Eva Riekert",de,NaN,9.783866e+12,"... und dann kam joselle kevin henkes, eva rie...",2
...,...,...,...,...,...,...,...,...
32577,"»Super«, schreit der Frieder, und die Oma kich...","»Super«, schreit der Frieder, und die Oma kich...",Gudrun Mebs,de,Der fünfte Band rund um Frieder und Oma und Fr...,9.783734e+12,"»super«, schreit der frieder, und die oma kich...",2
121474,Über die Grenze,Über die Grenze,Maja Lunde,de,Norwegen unter deutscher Besatzung 1942. Der z...,9.783825e+12,über die grenze maja lunde,2
121133,Über kurz oder lang,Über kurz oder lang,Marie-Aude Murail,de,Der neue großartige Roman von Marie-Aude Murai...,9.783104e+12,über kurz oder lang marie-aude murail,2
121135,Über kurz oder lang,Über kurz oder lang,"Marie-Aude Murail, Elinor Matt",de,NaN,9.783941e+12,"über kurz oder lang marie-aude murail, elinor ...",2


### need to go over diffDF/collectionDF mannually --> add to  sameDf / collectionDf if necessary

In [19]:
# mergedDf (sameDf + collectionDf)
merged = pd.concat([sameDf, collectionDf])
merged.sort_values('titleQueried')
merged.rename(columns = {'titleQueried':'title'}, inplace=True)
merged

,title,titleFound,author,language,description,isbn,key:title-author,collection
151865,#Instacat,#Instacat,Rae Earl,de,"Instagram, Eltern und andere Katastrophen ... ...",9.783846e+12,#instacat rae earl,2
4151,#Wir bleiben zuhause Kindermitmachbuch,#Wir bleiben zuhause Kindermitmachbuch,Frechverlag,de,NaN,9.783772e+12,#wir bleiben zuhause kindermitmachbuch frechve...,2
152172,'SIKARAN SHIELD LORD',Sikaran Shield Lord,Joanna Z. Ray,en,NaN,9.781858e+12,'sikaran shield lord' joanna z. ray,2
117696,(Un)arranged Marriage,(Un)arranged Marriage,Bali Rai,en,MANNY WANTS TO BE A FOOTBALLER. OR A POP STAR....,9.781446e+12,(un)arranged marriage bali rai,2
4931,... und dann kam Joselle,... und dann kam Joselle,"Kevin Henkes, Eva Riekert",de,NaN,9.783866e+12,"... und dann kam joselle kevin henkes, eva rie...",2
...,...,...,...,...,...,...,...,...
3338,tiptoi® Tiere im Garten,tiptoi® Dinosaurier,Annette Neubauer,de,Zusammen mit Paläontologe Florian Fossilius en...,9.783474e+12,tiptoi® tiere im garten annette neubauer,1
2311,your name. (light novel),Your Name. Visual Guide,Makoto Shinkai,de,NaN,9.783770e+12,your name. (light novel) makoto shinkai,1
2309,your name. (light novel),your name. 03,"Ranmaru Kotone, Makoto Shinkai",de,Um Mitsuha und ihr Dorf vor dem Einschlag des ...,9.783770e+12,"your name. (light novel) ranmaru kotone, makot...",1
2312,your name. (light novel),your name. Another Side:Earthbound (light novel),"Makoto Shinkai, Arata Kanoh",en,Mitsuha is a young girl living in a rural town...,9.780316e+12,"your name. (light novel) makoto shinkai, arata...",1


In [20]:
# add api data to items.csv data
addedData = pd.merge(book, merged, how="outer", on="title")
addedData.drop(['key:title-author_x', 'key:title-author_y'], axis=1, inplace=True)
addedData

,itemID,title,author_x,publisher,main topic,subtopics,titleFound,author_y,language,description,isbn,collection
0,26966,TRIPION minaccia dallo spazio,Stefano Grimaldi,Lulu.com,FL,[],NaN,NaN,NaN,NaN,NaN,NaN
1,24125,The Ultimate Vehicle Colouring Book for Kids,Chetna .,Westland Publications Limited,YBG,[],NaN,NaN,NaN,NaN,NaN,NaN
2,64846,!!,NaN,AUTHORHOUSE,FM,[],NaN,NaN,NaN,NaN,NaN,NaN
3,64745,"#4 Happy Birthday, Mallory!",Laurie Friedman,DARBY CREEK,YF,[5HKA],NaN,NaN,NaN,NaN,NaN,NaN
4,7780,#Basteln for Future,"Susanne Pypke, Naturschutzjugend NAJU",Frech Verlag GmbH,WF,"[5AF,Y]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
79599,21473,Übernatürliche Irreführung (Sasha Urban Serie 5),"Dima Zales, Anna Zaires",MOZAIKA LLC,FMX,[],NaN,NaN,NaN,NaN,NaN,NaN
79600,35275,Überschneidung zweier untersschiedlicher Welten,WErner Röschl,Books on Demand,FD,"[FD,FM]",NaN,NaN,NaN,NaN,NaN,NaN
79601,50552,Übersinnlicher Retter,Tina Folsom,Duboce Park Press,FRT,[],NaN,NaN,NaN,NaN,NaN,NaN
79602,5665,Übungsheft Handlettering: Übungsbuch Für Hand ...,"Sunshine Handlettering, Handlettering Heft In ...",INDEPENDENTLY PUBLISHED,YNV,[],NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
joinedData = pd.merge(book, merged, how="inner", on="title")
joinedData.drop(['key:title-author_x', 'key:title-author_y'], axis=1, inplace=True)
joinedData

,itemID,title,author_x,publisher,main topic,subtopics,titleFound,author_y,language,description,isbn,collection
0,4940,#Instacat,Rae Earl,Ars Edition GmbH,YFB,"[5AM,1DDU,YFN,YFQ,YFS]",#Instacat,Rae Earl,de,"Instagram, Eltern und andere Katastrophen ... ...",9.783846e+12,2
1,37807,#Wir bleiben zuhause Kindermitmachbuch,frechverlag,Frech Verlag GmbH,YBGC,"[5AF,5JA,5JB,YBGC,YBL,YZG]",#Wir bleiben zuhause Kindermitmachbuch,Frechverlag,de,NaN,9.783772e+12,2
2,20168,'SIKARAN SHIELD LORD',Reverend Joanna Z. Ray,Lulu.com,FL,[],Sikaran Shield Lord,Joanna Z. Ray,en,NaN,9.781858e+12,2
3,20168,'SIKARAN SHIELD LORD',Reverend Joanna Z. Ray,Lulu.com,FL,[],SIKARAN LIGHT,Joanna Ray,en,The Sikarans and humans are developing many in...,9.781327e+12,1
4,9363,(Un)arranged Marriage,Bali Rai,Penguin Random House Children's UK,YXHL,"[YFB,YXF]",(Un)arranged Marriage,Bali Rai,en,MANNY WANTS TO BE A FOOTBALLER. OR A POP STAR....,9.781446e+12,2
...,...,...,...,...,...,...,...,...,...,...,...,...
10752,56883,"»Super«, schreit der Frieder, und die Oma kich...",Gudrun Mebs,FISCHER KJB,YFM,"[5AG,YFN,YFQ]","»Super«, schreit der Frieder, und die Oma kich...",Gudrun Mebs,de,Der fünfte Band rund um Frieder und Oma und Fr...,9.783734e+12,2
10753,47970,Über die Grenze,Maja Lunde,cbt,YFT,"[5AL,5PGJ,1DNN,1DNS,YFT,YPJH,YXG,YXHB,YXR,YXZW...",Über die Grenze,Maja Lunde,de,Norwegen unter deutscher Besatzung 1942. Der z...,9.783825e+12,2
10754,62873,Über kurz oder lang,Marie-Aude Murail,FISCHER Taschenbuch,YFN,"[5AN,5AP,5AQ,1DDF-FR-GFA,YXD,YXT,YXV]",Über kurz oder lang,Marie-Aude Murail,de,Der neue großartige Roman von Marie-Aude Murai...,9.783104e+12,2
10755,62873,Über kurz oder lang,Marie-Aude Murail,FISCHER Taschenbuch,YFN,"[5AN,5AP,5AQ,1DDF-FR-GFA,YXD,YXT,YXV]",Über kurz oder lang,"Marie-Aude Murail, Elinor Matt",de,NaN,9.783941e+12,2


In [23]:
joinedData.to_csv('../Dataset/joinedData.csv')

# Preprocessing

In [224]:
joinedData = pd.read_csv("../Dataset/joinedData.csv")
joinedData.drop(['Unnamed: 0'], axis=1, inplace=True)

In [227]:
prep = joinedData.drop_duplicates('itemID', keep='first').sort_values('title')
prep = prep[["itemID", "title", "titleFound", "author_x", "author_y", "language", "collection", "isbn", "publisher", "description", "main topic", "subtopics"]]
prep["author_y"] = prep["author_y"].str.replace(pat=';.*', repl=r'', regex=True)

In [228]:
prep['key1'] = ((prep['title'] + ' '+ prep['author_x']).astype(str)).str.lower()
prep['key2'] = ((prep['title'] + ' '+ prep['author_y']).astype(str)).str.lower()
prep['itemID'] = prep['itemID'].astype(str)
prep['main topic'] = prep['main topic'].astype(str)

In [229]:
# prepocessing of collection == 1

col_1 = prep[prep['collection'] == 1]
keys = prep['key1']
col_11 = col_1[keys.isin(keys[keys.duplicated()])]
# a.to_excel('a.xlsx')
col_11 = col_11.groupby('key1').agg({'itemID': ' '.join,
                                    'title': 'first', 
                                    'author_x': 'first',
                                    'language':'first',
                                    'collection':'first',
                                    'isbn':'first',
                                    'publisher':'first',
                                    'description':'first',
                                    'main topic': ' '.join,
                                    'subtopics': ' '.join }).reset_index()

col_11.columns =['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']

for i in range(len(col_11)):
    col_11['subtopics'][i] = col_11['subtopics'][i].replace('[]', '')

idTemp = np.array(col_11["itemID"].tolist())
mainTopicTemp = np.array(col_11["main topic"].tolist())
subtopicTemp = np.array(col_11["subtopics"].tolist())
idList = []
mainTopicList = []
subtopicList = []

for i in range(len(col_11)):
    arrTemp1 = idTemp[i].split(' ')
    arrTemp2 = mainTopicTemp[i].split(' ')  
    arrTemp3 = subtopicTemp[i].replace(',',' ')
    arrTemp3 = arrTemp3.replace('[','')
    arrTemp3 = arrTemp3.replace(']','')
    arrTemp3 = arrTemp3.split(' ')
    
    idList.append(arrTemp1)
    mainTopicList.append(arrTemp2)
    subtopicList.append(arrTemp3)


for i in range(len(col_11)):
    idList[i] = list(set(idList[i]))
    mainTopicList[i] = list(set(mainTopicList[i]))
    subtopicList[i] = list(set(subtopicList[i]))

col_11['itemID'] = (idList)
col_11['main topic'] = (mainTopicList)
col_11['subtopics'] = (subtopicList)

# subtract col_11 from col_1 
for i in range(len(col_11)):
    for j in range(len(col_11['itemID'][i])):
        subtId = str(int(col_1[col_1['itemID'].isin([col_11['itemID'][i][j]])]['itemID']))
        idx = col_1[col_1['itemID'] == subtId].index
        col_1 = col_1.drop(idx)
col_1.head(3)

<ipython-input-229-300068b24da4>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  col_11 = col_1[keys.isin(keys[keys.duplicated()])]
<ipython-input-229-300068b24da4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col_11['subtopics'][i] = col_11['subtopics'][i].replace('[]', '')


,itemID,title,titleFound,author_x,author_y,language,collection,isbn,publisher,description,main topic,subtopics,key1,key2
7,2749,1000 Gefahren auf dem Tierhof,1000 Gefühle: Herzklopfen beim Schüleraustausch,Sonja Bullen,Sonja Bullen,de,1,9.783473e+12,Ravensburger Verlag,Lena freut sich wie verrückt auf ihren Schüler...,YFM,"[5AL,5AR,YFCA,YFM,YFQ]",1000 gefahren auf dem tierhof sonja bullen,1000 gefahren auf dem tierhof sonja bullen
8,18831,1000 Gefahren im Fußballstadion,1000 Gefahren auf dem Mars,Fabian Lenk,Fabian Lenk,de,1,9.783473e+12,Ravensburger Verlag,"Hier entscheidest du allein, wie dein Abenteue...",YFCA,"[5AL,5AR,YFR]",1000 gefahren im fußballstadion fabian lenk,1000 gefahren im fußballstadion fabian lenk
9,31991,1000 Gefahren in der versunkenen Stadt,1000 Gefahren im alten Rom,Fabian Lenk,Fabian Lenk,de,1,9.783474e+12,Ravensburger Verlag,NaN,YFC,[5AL],1000 gefahren in der versunkenen stadt fabian ...,1000 gefahren in der versunkenen stadt fabian ...


In [230]:
# same work for col_2

col_2 = prep[prep['collection'] == 2]
keys = prep['key2']
col_22 = col_2[keys.isin(keys[keys.duplicated()])]
col_22 = col_22.groupby('key2').agg({'itemID': ' '.join,
                                    'title': 'first', 
                                    'author_y': 'first',
                                    'language':'first',
                                    'collection':'first',
                                    'isbn':'first',
                                    'publisher':'first',
                                    'description':'first',
                                    'main topic': ' '.join,
                                    'subtopics': ' '.join }).reset_index()
# b.to_excel('b.xlsx')

col_22.columns =['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']

for i in range(len(col_22)):
    col_22['subtopics'][i] = col_22['subtopics'][i].replace('[]', '')
    
idTemp = np.array(col_22["itemID"].tolist())
mainTopicTemp = np.array(col_22["main topic"].tolist())
subtopicTemp = np.array(col_22["subtopics"].tolist())
idList = []
mainTopicList = []
subtopicList = []

for i in range(len(col_22)):
    arrTemp1 = idTemp[i].split(' ')
    arrTemp2 = mainTopicTemp[i].split(' ')
    
    arrTemp3 = subtopicTemp[i].replace(',',' ')
    arrTemp3 = arrTemp3.replace('[','')
    arrTemp3 = arrTemp3.replace(']','')
    arrTemp3 = arrTemp3.split(' ')
    
    idList.append(arrTemp1)
    mainTopicList.append(arrTemp2)
    subtopicList.append(arrTemp3)

for i in range(len(col_22)):
    idList[i] = list(set(idList[i]))
    mainTopicList[i] = list(set(mainTopicList[i]))
    subtopicList[i] = list(set(subtopicList[i]))
    
col_22['itemID'] = (idList)
col_22['main topic'] = (mainTopicList)
col_22['subtopics'] = (subtopicList)

# subtranct col_22 from col_2
for i in range(len(col_22)):
    for j in range(len(col_22['itemID'][i])):
        subtId = str(int(col_2[col_2['itemID'].isin([col_22['itemID'][i][j]])]['itemID']))
        idx = col_2[col_2['itemID'] == subtId].index
        col_2 = col_2.drop(idx)
col_2.head(3)

<ipython-input-230-c858ebd40af9>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  col_22 = col_2[keys.isin(keys[keys.duplicated()])]
<ipython-input-230-c858ebd40af9>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col_22['subtopics'][i] = col_22['subtopics'][i].replace('[]', '')


,itemID,title,titleFound,author_x,author_y,language,collection,isbn,publisher,description,main topic,subtopics,key1,key2
0,4940,#Instacat,#Instacat,Rae Earl,Rae Earl,de,2,9.783846e+12,Ars Edition GmbH,"Instagram, Eltern und andere Katastrophen ... ...",YFB,"[5AM,1DDU,YFN,YFQ,YFS]",#instacat rae earl,#instacat rae earl
1,37807,#Wir bleiben zuhause Kindermitmachbuch,#Wir bleiben zuhause Kindermitmachbuch,frechverlag,Frechverlag,de,2,9.783772e+12,Frech Verlag GmbH,NaN,YBGC,"[5AF,5JA,5JB,YBGC,YBL,YZG]",#wir bleiben zuhause kindermitmachbuch frechve...,#wir bleiben zuhause kindermitmachbuch frechve...
2,20168,'SIKARAN SHIELD LORD',Sikaran Shield Lord,Reverend Joanna Z. Ray,Joanna Z. Ray,en,2,9.781858e+12,Lulu.com,NaN,FL,[],'sikaran shield lord' reverend joanna z. ray,'sikaran shield lord' joanna z. ray


In [231]:
# merge 
col_1.rename(columns = {'key1':'key'}, inplace=True)
col_1.rename(columns = {'author_x':'author'}, inplace=True)
col_1.drop(['titleFound', 'author_y', 'key2'], axis=1, inplace=True)
col_1 = col_1[['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']]

col_2.rename(columns = {'key2':'key'}, inplace=True)
col_2.rename(columns = {'author_y':'author'}, inplace=True)
col_2.drop(['titleFound', 'author_x', 'key1'], axis=1, inplace=True)
col_2 = col_2[['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']]

newCol1 = pd.concat([col_1, col_11])
newCol2 = pd.concat([col_2, col_22])
processedData = pd.concat([newCol1, newCol2])

In [232]:
processedData.drop(['key'], axis=1, inplace=True)
processedData.head(3)
processedData.to_csv('../Dataset/processedData.csv')

In [212]:
processedData.head(3)

,itemID,title,author,language,collection,isbn,publisher,description,main topic,subtopics
7,2749,1000 Gefahren auf dem Tierhof,Sonja Bullen,de,1,9.783473e+12,Ravensburger Verlag,Lena freut sich wie verrückt auf ihren Schüler...,YFM,"[5AL,5AR,YFCA,YFM,YFQ]"
8,18831,1000 Gefahren im Fußballstadion,Fabian Lenk,de,1,9.783473e+12,Ravensburger Verlag,"Hier entscheidest du allein, wie dein Abenteue...",YFCA,"[5AL,5AR,YFR]"
9,31991,1000 Gefahren in der versunkenen Stadt,Fabian Lenk,de,1,9.783474e+12,Ravensburger Verlag,NaN,YFC,[5AL]
